# Ансамбль регрессионных моделей

1. Загрузить подготовленные данные
2. Собрать два набора моделей: по дате (праздники, дни недели и т.д.) и по погоде
3. Провести 10 разбиений данных на обучающие/проверочные и выявить оптимальные веса моделей для каждого часа для каждого здания.
4. Вычислить оптимизированную метрику качества для ансамблей моделей.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Загрузка данных 20 зданий из HDF5

In [2]:
energy = pd.read_hdf("../data/out/energy.0-20.h5", "energy")
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-null  float16       
 12  air_temperature_diff_2  175680

## Набор параметров для каждой модели

In [3]:
lr_weather_columns = [
    "meter_reading_log",
    "hour",
    "building_id",
    "air_temperature",
    "dew_temperature",
    "sea_level_pressure",
    "wind_speed",
    "air_temperature_diff_1",
    "air_temperature_diff_2",
    "cloud_coverage",
]
lr_days_columns = [
    "meter_reading_log",
    "hour",
    "building_id",
    "is_holiday",
]

for wday in range(0, 7):
    lr_days_columns.append(f"is_wday {str(wday)}")
for week in range(1, 54):
    lr_days_columns.append(f"is_w {str(week)}")
for month in range(1, 13):
    lr_days_columns.append(f"is_m {str(month)}")

hours = range(0, 24)
buildings = range(0, energy["building_id"].max() + 1)

## Функция для вычисления качества моделей

In [4]:
def calculate_model(x, df_lr, lr_columns):
    lr = -1
    model = df_lr[x.building_id][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i, col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns) - 3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) - np.log(1 + lr))**2

    return x

## Функция разделения данных

In [5]:
def train_model(df, columns):
    df_train_lr = pd.DataFrame(df, columns=columns)
    df_lr = [[] for _ in range(len(buildings))]
    for building in buildings:
        df_lr[building] = [[] for _ in range(len(hours))]
        df_train_b = df_train_lr[df_train_lr["building_id"] == building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b["hour"] == hour]
            y = df_train_bh["meter_reading_log"]
            x = df_train_bh.drop(
                labels=[
                    "meter_reading_log",
                    "hour",
                    "building_id"
                ],
                axis=1
            )
            model = LinearRegression(fit_intercept=False).fit(x, y)
            df_lr[building][hour] = model.coef_
            df_lr[building][hour] = np.append(df_lr[building][hour], model.intercept_)
    return df_lr  

## Вычисление качества моделей

In [6]:
def calculate_weights_model(df_test, df_train, lr_columns):
    df_test = df_test.apply(
        calculate_model,
        axis=1,
        result_type="expand",
        df_lr=train_model(df_train, lr_columns),
        lr_columns=lr_columns
    )

    return pd.Series(df_test.groupby(["hour", "building_id"])["meter_reading_lr_q"].sum())

In [7]:
def calculate_weights():
    df_train, df_test = train_test_split(energy[energy["meter_reading"] > 0], test_size=0.2)
    return (calculate_weights_model(df_test, df_train, lr_weather_columns),
            calculate_weights_model(df_test, df_train, lr_days_columns))


## Расчитать оптимальные веса для каждого часа и здания

10 раз разбить исходный набор данных на обучающую/тестовую выборку, рассчитаем в каждом случае значение ошибки для каждого здания и часа.
Сформировать список весов: 1 - учитываем регрессию по дням недели, 0 - учитываем регрессию по погоде.

In [8]:
import warnings
  
# suppress warnings
warnings.filterwarnings('ignore')
  

weights_weather = []
weights_days = []
for i in range(0, 10):
    print(f"Расчет весов ансамбля, итереция #{i}")

    weights_weather_model, weights_days_model = calculate_weights()
    if len(weights_weather) > 0:
        weights_weather = weights_weather + weights_weather_model
    else:
        weights_weather = weights_weather_model
    
    if len(weights_days) > 0:
        weights_days = weights_days + weights_days_model
    else:
        weights_days = weights_days_model
    
    weights = [0 for _ in range(len(buildings))]
    for b in buildings:
        weights[b] = [0 for _ in range(len(hours))]
        for h in hours:
            if weights_weather.loc[h].at[b] > weights_days.loc[h].at[b]:
                weights[b][h] = 1
print(weights)

Расчет весов ансамбля, итереция #0
Расчет весов ансамбля, итереция #1
Расчет весов ансамбля, итереция #2
Расчет весов ансамбля, итереция #3
Расчет весов ансамбля, итереция #4
Расчет весов ансамбля, итереция #5
Расчет весов ансамбля, итереция #6
Расчет весов ансамбля, итереция #7
Расчет весов ансамбля, итереция #8
Расчет весов ансамбля, итереция #9
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

## Разделить данные на обучающие и тестовые

In [9]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)

## Обучить модели линейной регрессии по дате/погоде

In [10]:
energy_lr_days = train_model(energy_train, lr_days_columns)
energy_lr_weather = train_model(energy_train, lr_weather_columns)

## Рассчет финального качества ансамбля
Если вес 1, то считать регрессию по дням недели, если 0 - то по погоде

In [11]:
def calculate_model_ensemble(x, model, columns):
    lr = -1
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i, col in enumerate(columns[3:])])
        lr += model[len(columns) - 3]
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0
    return lr

In [12]:
def calculate_models_ensemble(x):
    lr_d = calculate_model_ensemble(
        x,
        energy_lr_days[x.building_id][x.hour],
        lr_days_columns
    )
    lr_w = calculate_model_ensemble(
        x,
        energy_lr_weather[x.building_id][x.hour],
        lr_weather_columns
    )
    if weights[x.building_id][x.hour] == 1:
        lr = lr_d
    else:
        lr = lr_w
    lr_sum = (lr_w + lr_d)/2
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) - np.log(1 + lr))**2
    x["meter_reading_sum_q"] = (np.log(x.meter_reading + 1) - np.log(1 + lr_sum))**2

    return x

In [13]:
energy_test = energy_test.apply(calculate_models_ensemble, axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
energy_test_sum_rmsle = np.sqrt(energy_test["meter_reading_sum_q"].sum() / len(energy_test))
print(
    f"Качество ансамбля, 20 зданий: {energy_test_lr_rmsle}, округленно - {round(energy_test_lr_rmsle, 1)}")
print(
    f"Качество ансамбля суммы, 20 зданий: {energy_test_sum_rmsle}, округленно - {round(energy_test_sum_rmsle, 1)}")


Качество ансамбля, 20 зданий: 0.22868132801883848, округленно - 0.2
Качество ансамбля суммы, 20 зданий: 0.20168215570007034, округленно - 0.2
